In [1]:
from glob import glob
import os
from pathlib import Path
from enum import Enum
import json
import pandas as pd
import seaborn as sns

In [2]:
UNATES_LOOKUP_PATH = '../experiments/find-unates/'

# Load Experiment Results

In [3]:
class Status(Enum):
    ERROR = "Error"
    SUCCESS = "Success"
    NOT_FOUND = "Not Found"
    
    def __repr__(self):
        return str(self.value)
    
    def __str__(self):
        return str(self.value)

In [4]:
def get_all_benchmarks(path):
    hoa_files = glob(os.path.join(path, "*.json"))
    names = [
        Path(f).stem
        for f in hoa_files
    ]
    return names

print("Total benchmark of Unates" , len(get_all_benchmarks(UNATES_LOOKUP_PATH)))

Total benchmark of Unates 202


In [1]:
def benchmark_unate_loader(path: str):
    def load_benchmark(name: str):
        base = {
            'Name': name,
        }
        
        json_path = os.path.join(path, name + ".json")
        
        if not os.path.exists(json_path):
            return {
                **base,
                'Status': Status.NOT_FOUND
            }
        
        with open(json_path, 'r') as f:
            content = f.readlines()
            try:
                benchmark_json = json.loads(content[0])
            except Exception as e:
                return {
                    **base,
                    'Status': Status.ERROR,
                    'Error': content
                }
        
        is_completed = benchmark_json['is_completed']
        total_duration = benchmark_json['total_time']
        is_automaton_build = benchmark_json['automaton']['is_built']
        if is_automaton_build:
            automaton_build_duration = benchmark_json['automaton']['build_duration']
            total_states = benchmark_json['automaton']['total_states']
            total_state_check_duration = {
                state: 0
                for state in range(total_states)
            }
            search_unates_duration = total_duration - automaton_build_duration
        else:
            automaton_build_duration, total_states = None, None
            total_state_check_duration = {}
            search_unates_duration = None
        
        total_output_vars = len(benchmark_json['output_vars'])
        
        variables_with_unates = set([])
        total_removable_edges = 0
        
        for tested_var in benchmark_json['tested_unates']:
            total_state_check_duration[tested_var['state']] += tested_var['duration']
            
            if not tested_var['is_unate']:
                continue
            
            variables_with_unates.add(tested_var['variable'])
            total_removable_edges += tested_var['total_removable_edges']
        
        return {
            **base,
            'Status': Status.SUCCESS,
            'Is Completed': is_completed,
            'Is Automaton Built': is_automaton_build,
            'Total Duration': total_duration,
            'Automaton Build Duration': automaton_build_duration,
            'Search Unate Durations': search_unates_duration,
            'Total States': total_states,
            'Total Output Vars': total_output_vars,
            'Total Unates Vars': len(list(variables_with_unates)),
            'Vars with Unates': list(variables_with_unates),
            'Removable Edges': total_removable_edges,
            'Test States Duration': total_state_check_duration,
        }

    return load_benchmark

In [7]:
unates_loader = benchmark_unate_loader(UNATES_LOOKUP_PATH)
unates_df = pd.DataFrame([
    unates_loader(name)
    for name in get_all_benchmarks(UNATES_LOOKUP_PATH)
])
unates_df.to_csv('unates.csv', index=False)
unates_df.head(5)

,Name,Status,Is Completed,Is Automaton Built,Total Duration,Automaton Build Duration,Search Unate Durations,Total States,Total Output Vars,Vars with Unates,Removable Edges,Test States Duration,Error
0,ltl2dpa20,Success,True,True,27.0,22.0,5.0,10.0,2.0,[p1],0.0,"{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",NaN
1,TwoCountersInRangeA2,Success,True,True,318140.0,5203.0,312937.0,6.0,8.0,[],0.0,"{0: 159507, 1: 29131, 2: 32066, 3: 30386, 4: 3...",NaN
2,lilydemo24,Success,True,True,42.0,17.0,25.0,14.0,4.0,"[b2r_req1, b2s_ack0, b2r_req0, b2s_ack1]",149.0,"{0: 7, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: ...",NaN
3,OneCounterGuiA9,Success,True,True,2437011.0,19406.0,2417605.0,9.0,9.0,[],0.0,"{0: 1483057, 1: 23796, 2: 25369, 3: 24890, 4: ...",NaN
4,ltl2dba24,Success,True,True,10.0,9.0,1.0,4.0,1.0,[acc],5.0,"{0: 0, 1: 0, 2: 0, 3: 0}",NaN
